In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df = pd.read_csv('insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [4]:
# so lets look at the distribtion of people who smoke and those who do not

In [5]:
import plotly.express as px

In [6]:
fig = px.histogram(df, x="sex", color="smoker", title="number of smokers")

In [7]:
fig.show()

In [8]:
# so now we will replace the value of 'sex' and 'smoker' with 0 and 1 respectively

 -->ENCODING

In [9]:
df['sex'] = df['sex'].map({'female':0 , 'male':1})
df['smoker'] = df['smoker'].map({'no':0 , 'yes':1})
df['region'] = df['region'].map({'southwest':0 , 'southeast':1, 'northwest':2, 'northeast':3})
print(df.head())

   age  sex     bmi  children  smoker  region      charges
0   19    0  27.900         0       1       0  16884.92400
1   18    1  33.770         1       0       1   1725.55230
2   28    1  33.000         3       0       1   4449.46200
3   33    1  22.705         0       0       2  21984.47061
4   32    1  28.880         0       0       2   3866.85520


In [10]:
# now lets have a look at the distribution of the regions where people are living acc to the data 

In [11]:
import plotly.express as px


In [12]:
pie = df['region'].value_counts()
regions = pie.index
population = pie.values
fig = px.pie(df , values = population , names = regions)
fig.show()

In [13]:
# now lets have a look at the corerlation between the features of this dataset

In [14]:
print(df.corr())

               age       sex       bmi  children    smoker    region   charges
age       1.000000 -0.020856  0.109272  0.042469 -0.025019 -0.002127  0.299008
sex      -0.020856  1.000000  0.046371  0.017163  0.076185 -0.004588  0.057292
bmi       0.109272  0.046371  1.000000  0.012759  0.003750 -0.157566  0.198341
children  0.042469  0.017163  0.012759  1.000000  0.007673 -0.016569  0.067998
smoker   -0.025019  0.076185  0.003750  0.007673  1.000000  0.002181  0.787251
region   -0.002127 -0.004588 -0.157566 -0.016569  0.002181  1.000000  0.006208
charges   0.299008  0.057292  0.198341  0.067998  0.787251  0.006208  1.000000


In [15]:

# now lets move to training a machine learning model for the task of predicting health insurance 

In [16]:
x = np.array(df.drop(['age', 'sex', 'bmi', 'smoker'], axis=1))
y = np.array(df['age'])

In [17]:
# now splitting the data into training and testing data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [18]:
# now we will use Random Forest Regressor to train the model
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(x_train, y_train)
# now we will test the model on the test data
y_pred = rfr.predict(x_test)
# now we will check the accuracy of the model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('R2 Score:', r2)




Mean Absolute Error: 3.175335820895522
Mean Squared Error: 35.486065298507455
R2 Score: 0.817327595564949


In [19]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score before fine -tuning: {r2:.2f}")

R^2 Score before fine -tuning: 0.82


In [20]:
data = pd.DataFrame({'predicted premium account': y_pred})
print(data.head())

   predicted premium account
0                      46.62
1                      35.99
2                      61.11
3                      46.13
4                      20.23


In [21]:
# now we will do fine tuning of the model using GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import    RandomForestRegressor

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
model = RandomForestRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(x_train, y_train)
best_model = grid_search.best_estimator_
best_model.fit(x_train, y_train)
y_pred = best_model.predict(x_test)
r2 = r2_score(y_test, y_pred)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


In [22]:
print(f"R^2 Score after fine tuning: {r2}")
print(f"Best parameters: {grid_search.best_params_}")

R^2 Score after fine tuning: 0.8288223458212776
Best parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
